### Set Up

In [ ]:
import requests

API_URL = 'https://api.loyverse.com/v1.0/items'
MODIFIER_URL = 'https://api.loyverse.com/v1.0/modifiers'
API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token

# Define the headers with the API token
headers = {
    'Authorization': f'Bearer {API_TOKEN}'
}

# Define a function to fetch items with pagination
def fetch_items_by_category(api_url, headers):
    all_items = []
    params = {
        'limit': 250
    }
    while True:
        response = requests.get(api_url, headers=headers, params=params)
        if response.status_code == 200:
            items = response.json().get('items', [])
            all_items.extend(items)
            cursor = response.json().get('cursor')
            if cursor:
                params['cursor'] = cursor  # Use the cursor to fetch the next page
            else:
                break
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            break
    return all_items



### Add An Item

In [ ]:
# test add item to the store
import requests
import json

def create_or_update_item(api_token, item_name, description, category_id, price, stores_info):
    """
    Create or update an item in Loyverse with specific pricing and availability for multiple stores.

    Parameters:
    api_token (str): API token for authentication.
    item_name (str): Name of the item.
    description (str): Description of the item.
    category_id (str): UUID of the category to which the item belongs.
    price (float): Price of the item.
    stores_info (list): List of dictionaries containing store-specific pricing and availability details.

    Returns:
    dict: Response from the API as a JSON object.
    """
    API_URL = 'https://api.loyverse.com/v1.0/items'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Content-Type': 'application/json'
    }

    # Construct the JSON data payload for the API request
    item_data = {
        "item_name": item_name,
        "description": description,
        "category_id": category_id,
        "track_stock": False,  # Assuming stock tracking is not required
        "sold_by_weight": False,
        "is_composite": False,
        "use_production": False,
        "variants": [
            {
                "default_pricing_type": "FIXED",
                "default_price": price,
                "stores": stores_info
            }
        ],
        "form": "SQUARE",
        "color": "GREY"
    }

    # Make the POST request to create or update the item
    response = requests.post(API_URL, headers=headers, json=item_data)
    return response.json()

def add_items_to_loyverse(api_token, items_data, category_ids, store_ids, store_availability):
    """
    Loop through items data to create or update items with specific store settings.

    Parameters:
    api_token (str): API token.
    items_data (dict): Dictionary of categories and their items.
    category_ids (dict): Dictionary mapping category names to IDs.
    store_ids (list): List of store IDs.
    store_availability (dict): Availability of items per store.
    """
    for category_name, items in items_data.items():
        category_id = category_ids[category_name]
        for item in items:
            stores_info = [
                {
                    "store_id": store_ids[0],
                    "pricing_type": "FIXED",
                    "price": item['price'],
                    "available_for_sale": store_availability[store_ids[0]]
                },
                {
                    "store_id": store_ids[1],
                    "pricing_type": "FIXED",
                    "price": item['price'],
                    "available_for_sale": store_availability[store_ids[1]]
                }
            ]
            response = create_or_update_item(api_token, item['name'], item['description'], category_id, item['price'], stores_info)
            print(f"Added/Updated {item['name']} in category {category_name}: {response}")

# Placeholder for category IDs
category_ids = {
    "Signature Cocktail": "ff40ed04-4d56-4d46-a9e4-eac8c24c862c",
    "Signature Mocktail": "99246943-5a07-4498-aefb-e8d202868e12",
    "Classic Cocktail": "2dd090f9-5a23-48dc-90b4-85447097d847",
    "Classic & Other Martini": "391d2cf2-3ce0-4606-8b0c-f5f28bf1ea4e",
    "Favorite": "41b8a1c5-967f-4975-a73f-0f375baf05af",
    "Mimosas": "456707cb-b6b1-4845-a58c-8c92c7740118",
    "Whisky": "958b84c4-aa43-4198-b06c-9cdd41518503",
    "Cognac & Brandy": "fd79887b-4c6e-4af1-9ea7-e8947c318dda",
    "Gin": "a79fbaef-ed90-41a6-a5cf-82581de52cf6",
    "Rum": "d2e7a7c3-ed39-4a05-8cd0-36354017c087",
    "Vodka": "97976a29-ec7c-4955-a38f-2a05dd909e0b",
    "Tequila": "55e67206-6c2c-4e06-8d40-42361e33ec59",
    "Liqueur": "bb2add66-cb92-40d6-b926-e6d26ef5fa38",
    "House Wine [By the Glass]": "e768b2fa-4aa3-492e-84ab-d7853431c693",
    "Red Wine/Bottle": "80c5657e-3aae-48ce-ae99-9d6f95fbf840",
    "White Wine/Bottle": "691669cf-9206-407d-8e3f-73d8ba6fa32b",
    "Moscato/Bottle": "0cfced9a-f7ad-4122-a253-0eb3dfef073a",
    "Sparkling Wine/Bottle": "b939c9fd-bbac-4f0e-8e26-fb477adb6e9e",
    "Beer & Soft Drink": "e67604d4-39c4-48ba-8ae3-22571a15a746",
    "Sparkling & Smoothie": "e9110298-cd94-483d-8dff-6240c4fcdbf1",
    "Whisky & Liquor Collection Custom Drink": "dfadedd2-38a9-4f84-bc30-cb2842f00f4e",
    "Cognac & Brandy Custom Drink": "cae31596-1a44-4994-87e4-7c547d3041ce",
    "Gin Custom Drink": "d11568ee-71c4-424d-b3e3-c9d417e4693a",
    "Rum Custom Drink": "bcd0a1d9-64c6-431b-a66e-f4b36e4509bf",
    "Vodka Custom Drink": "8d801ff9-77a8-4ff8-a94f-cc4da660f28f",
    "Tequila Custom Drink": "495f385a-0d82-43ff-bbdb-bad5906902d2",
    "Liqueur Custom Drink": "d94a15f0-0676-41f8-9892-e6911b2cfc27",
    "American Inspired" : "195dae33-5bb1-474d-8368-329209fbf106",
    
}


# Item data structured by category
items_data = {
    
    "American Inspired" : [
    {
        "name": "Qulture Creamy Cajun Pasta Alfredo",
        "price": 350.00,
        "description": "Served with a side of garlic French bread"
    },
    {
        "name": "Creamy Black Garlic Mushroom Pasta",
        "price": 450.00,
        "description": ""
    },
    {
        "name": "Creamy Tuscan Salmon",
        "price": 550.00,
        "description": "Grilled salmon with sautéed garlic, cherry tomatoes, spinach, in a white wine cream sauce over spaghetti noodles"
    },
    {
        "name": "Fried Snow Crab Legs",
        "price": 0.00,  # Indicative placeholder as the price is market-dependent
        "description": "Market Price per Kg"
    }
]
}




# Example store IDs and their sale availability
store_ids = ['c4da6b6a-483f-4aec-ab62-8dbcbff26d93', '17d2d59c-227a-4929-92bb-fa67a57247c2']
store_availability = {
    'c4da6b6a-483f-4aec-ab62-8dbcbff26d93': True,
    '17d2d59c-227a-4929-92bb-fa67a57247c2': False
}
api_token = API_TOKEN

# Call the function with the necessary parameters
add_items_to_loyverse(api_token, items_data, category_ids, store_ids, store_availability)

### Create category

In [ ]:
# Test acreate category
import requests
import json

def create_or_update_category(name, color='GREY', category_id=None):
    """
    Create or update a category in Loyverse.

    :param name: Name of the category (1 to 64 characters).
    :param color: Color of the category, defaults to 'GREY'.
    :param category_id: Optional UUID string to update an existing category.
    :return: Response from the API.
    """
    # API endpoint variable
    API_URL_CAT = 'https://api.loyverse.com/v1.0/categories'
    API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token
    
    # Headers with authorization and content type
    headers = {
        'Authorization': f'Bearer {API_TOKEN}',
        'Content-Type': 'application/json'
    }
    
    # Data to be sent to API
    data = {
        'name': name.upper(),
        'color': color.upper()
    }
    
    # If category_id is provided, set it in data to update an existing category
    if category_id:
        data['id'] = category_id
    
    # Make POST request to create or update category
    response = requests.post(API_URL_CAT, headers=headers, json=data)
    
    # Return the response as a JSON object
    return response.json()

# Example of creating a new category
new_category = create_or_update_category('Signature Cocktail', 'red')
print(new_category)





In [ ]:
import requests
import json

def create_or_update_category(name, color='GREY', category_id=None):
    """
    Create or update a category in Loyverse.

    :param name: Name of the category (1 to 64 characters).
    :param color: Color of the category, defaults to 'GREY'.
    :param category_id: Optional UUID string to update an existing category.
    :return: Response from the API.
    """
    API_URL_CAT = 'https://api.loyverse.com/v1.0/categories'
    API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token
    
    headers = {
        'Authorization': f'Bearer {API_TOKEN}',
        'Content-Type': 'application/json'
    }
    
    data = {
        'name': name.upper(),
        'color': color.upper()
    }
    
    if category_id:
        data['id'] = category_id
    
    response = requests.post(API_URL_CAT, headers=headers, json=data)
    return response.json()

# List of categories to be created
categories_to_create = [
    "Signature Cocktail",
    "Signature Mocktail",
    "Classic cocktail",
    "Classic & Other Martini",
    "Favorite",
    "Mimosas",
    "Whisky",
    "Cognac & Brandy",
    "Gin",
    "Rum",
    "Vodka",
    "Tequila",
    "Liqueur",
    "House wine [ by the glass ]",
    "Red wine/Bottle",
    "White wine/Bottle",
    "Moscato/Bottle",
    "Sparkling wine/Bottle",
    "Beer&Solf Drink",
    "Sparkling&Smoothie",
    "Whisky&Liquor Collection_Custom_Drink",
    "Cognac & Brandy_Custom_Drink",
    "Gin_Custom_Drink",
    "Rum_Custom_Drink",
    "Vodka_Custom_Drink",
    "Tequila_Custom_Drink",
    "Liqueur_Custom_Drink"
]

# Loop to create each category
for category_name in categories_to_create:
    result = create_or_update_category(category_name)
    print(f"Created/Updated category: {result['name']} with ID: {result['id']}")



- Created/Updated category: **SIGNATURE COCKTAIL** with ID: `ff40ed04-4d56-4d46-a9e4-eac8c24c862c`
- Created/Updated category: **SIGNATURE MOCKTAIL** with ID: `99246943-5a07-4498-aefb-e8d202868e12`
- Created/Updated category: **CLASSIC COCKTAIL** with ID: `2dd090f9-5a23-48dc-90b4-85447097d847`
- Created/Updated category: **CLASSIC & OTHER MARTINI** with ID: `391d2cf2-3ce0-4606-8b0c-f5f28bf1ea4e`
- Created/Updated category: **FAVORITE** with ID: `41b8a1c5-967f-4975-a73f-0f375baf05af`
- Created/Updated category: **MIMOSAS** with ID: `456707cb-b6b1-4845-a58c-8c92c7740118`
- Created/Updated category: **WHISKY** with ID: `958b84c4-aa43-4198-b06c-9cdd41518503`
- Created/Updated category: **COGNAC & BRANDY** with ID: `fd79887b-4c6e-4af1-9ea7-e8947c318dda`
- Created/Updated category: **GIN** with ID: `a79fbaef-ed90-41a6-a5cf-82581de52cf6`
- Created/Updated category: **RUM** with ID: `d2e7a7c3-ed39-4a05-8cd0-36354017c087`
- Created/Updated category: **VODKA** with ID: `97976a29-ec7c-4955-a38f-2a05dd909e0b`
- Created/Updated category: **TEQUILA** with ID: `55e67206-6c2c-4e06-8d40-42361e33ec59`
- Created/Updated category: **LIQUEUR** with ID: `bb2add66-cb92-40d6-b926-e6d26ef5fa38`
- Created/Updated category: **HOUSE WINE [ BY THE GLASS ]** with ID: `e768b2fa-4aa3-492e-84ab-d7853431c693`
- Created/Updated category: **RED WINE/BOTTLE** with ID: `80c5657e-3aae-48ce-ae99-9d6f95fbf840`
- Created/Updated category: **WHITE WINE/BOTTLE** with ID: `691669cf-9206-407d-8e3f-73d8ba6fa32b`
- Created/Updated category: **MOSCATO/BOTTLE** with ID: `0cfced9a-f7ad-4122-a253-0eb3dfef073a`
- Created/Updated category: **SPARKLING WINE/BOTTLE** with ID: `b939c9fd-bbac-4f0e-8e26-fb477adb6e9e`
- Created/Updated category: **BEER&SOLF DRINK** with ID: `e67604d4-39c4-48ba-8ae3-22571a15a746`
- Created/Updated category: **SPARKLING&SMOOTHIE** with ID: `e9110298-cd94-483d-8dff-6240c4fcdbf1`
- Created/Updated category: **WHISKY&LIQUOR COLLECTION_CUSTOM_DRINK** with ID: `dfadedd2-38a9-4f84-bc30-cb2842f00f4e`
- Created/Updated category: **COGNAC & BRANDY_CUSTOM_DRINK** with ID: `cae31596-1a44-4994-87e4-7c547d3041ce`
- Created/Updated category: **GIN_CUSTOM_DRINK** with ID: `d11568ee-71c4-424d-b3e3-c9d417e4693a`
- Created/Updated category: **RUM_CUSTOM_DRINK** with ID: `bcd0a1d9-64c6-431b-a66e-f4b36e4509bf`
- Created/Updated category: **VODKA_CUSTOM_DRINK** with ID: `8d801ff9-77a8-4ff8-a94f-cc4da660f28f`
- Created/Updated category: **TEQUILA_CUSTOM_DRINK** with ID: `495f385a-0d82-43ff-bbdb-bad5906902d2`
- Created/Updated category: **LIQUEUR_CUSTOM_DRINK** with ID: `d94a15f0-0676-41f8-9892-e6911b2cfc27`


Created/Updated category: SIGNATURE COCKTAIL with ID: ff40ed04-4d56-4d46-a9e4-eac8c24c862c

Created/Updated category: SIGNATURE MOCKTAIL with ID: 99246943-5a07-4498-aefb-e8d202868e12

Created/Updated category: CLASSIC COCKTAIL with ID: 2dd090f9-5a23-48dc-90b4-85447097d847

Created/Updated category: CLASSIC & OTHER MARTINI with ID: 391d2cf2-3ce0-4606-8b0c-f5f28bf1ea4e

Created/Updated category: FAVORITE with ID: 41b8a1c5-967f-4975-a73f-0f375baf05af

Created/Updated category: MIMOSAS with ID: 456707cb-b6b1-4845-a58c-8c92c7740118

Created/Updated category: WHISKY with ID: 958b84c4-aa43-4198-b06c-9cdd41518503

Created/Updated category: COGNAC & BRANDY with ID: fd79887b-4c6e-4af1-9ea7-e8947c318dda

Created/Updated category: GIN with ID: a79fbaef-ed90-41a6-a5cf-82581de52cf6

Created/Updated category: RUM with ID: d2e7a7c3-ed39-4a05-8cd0-36354017c087

Created/Updated category: VODKA with ID: 97976a29-ec7c-4955-a38f-2a05dd909e0b

Created/Updated category: TEQUILA with ID: 55e67206-6c2c-4e06-8d40-42361e33ec59

Created/Updated category: LIQUEUR with ID: bb2add66-cb92-40d6-b926-e6d26ef5fa38

Created/Updated category: HOUSE WINE [ BY THE GLASS ] with ID: e768b2fa-4aa3-492e-84ab-d7853431c693

Created/Updated category: RED WINE/BOTTLE with ID: 80c5657e-3aae-48ce-ae99-9d6f95fbf840

Created/Updated category: WHITE WINE/BOTTLE with ID: 691669cf-9206-407d-8e3f-73d8ba6fa32b

Created/Updated category: MOSCATO/BOTTLE with ID: 0cfced9a-f7ad-4122-a253-0eb3dfef073a

Created/Updated category: SPARKLING WINE/BOTTLE with ID: b939c9fd-bbac-4f0e-8e26-fb477adb6e9e

Created/Updated category: BEER&SOLF DRINK with ID: e67604d4-39c4-48ba-8ae3-22571a15a746

Created/Updated category: SPARKLING&SMOOTHIE with ID: e9110298-cd94-483d-8dff-6240c4fcdbf1

Created/Updated category: WHISKY&LIQUOR COLLECTION_CUSTOM_DRINK with ID: dfadedd2-38a9-4f84-bc30-cb2842f00f4e

Created/Updated category: COGNAC & BRANDY_CUSTOM_DRINK with ID: cae31596-1a44-4994-87e4-7c547d3041ce

Created/Updated category: GIN_CUSTOM_DRINK with ID: d11568ee-71c4-424d-b3e3-c9d417e4693a

Created/Updated category: RUM_CUSTOM_DRINK with ID: bcd0a1d9-64c6-431b-a66e-f4b36e4509bf

Created/Updated category: VODKA_CUSTOM_DRINK with ID: 8d801ff9-77a8-4ff8-a94f-cc4da660f28f

Created/Updated category: TEQUILA_CUSTOM_DRINK with ID: 495f385a-0d82-43ff-bbdb-bad5906902d2

Created/Updated category: LIQUEUR_CUSTOM_DRINK with ID: d94a15f0-0676-41f8-9892-e6911b2cfc27

#### Get categories

In [ ]:
import requests

# API details
API_URL_CAT = 'https://api.loyverse.com/v1.0/categories'
API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token

# Define the headers with the API token
headers = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

def fetch_categories(api_url, headers):
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json().get('categories', [])
    else:
        print(f"Failed to fetch categories. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return []

# Fetch categories
categories = fetch_categories(API_URL_CAT, headers)
categories


[{'id': '195dae33-5bb1-474d-8368-329209fbf106',
  'name': 'AMERICAN INSPIRED',
  'color': 'GREY',
  'created_at': '2024-08-08T09:09:26.000Z',
  'deleted_at': None},
 {'id': 'dfa06e6f-68bb-4eb3-816b-d17363255888',
  'name': 'STARTER/APPETIZER',
  'color': 'GREY',
  'created_at': '2024-08-08T09:09:10.000Z',
  'deleted_at': None},
 {'id': 'd94a15f0-0676-41f8-9892-e6911b2cfc27',
  'name': 'LIQUEUR_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:12.000Z',
  'deleted_at': None},
 {'id': '495f385a-0d82-43ff-bbdb-bad5906902d2',
  'name': 'TEQUILA_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:11.000Z',
  'deleted_at': None},
 {'id': '8d801ff9-77a8-4ff8-a94f-cc4da660f28f',
  'name': 'VODKA_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:10.000Z',
  'deleted_at': None},
 {'id': 'bcd0a1d9-64c6-431b-a66e-f4b36e4509bf',
  'name': 'RUM_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:10.000Z',
  'deleted_at': None},
 {'id': 'd11568ee-71c4-424d-b3e3-c9d417e4693a',
  'name': 'GIN_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:09.000Z',
  'deleted_at': None},
 {'id': 'cae31596-1a44-4994-87e4-7c547d3041ce',
  'name': 'COGNAC & BRANDY_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:08.000Z',
  'deleted_at': None},
 {'id': 'dfadedd2-38a9-4f84-bc30-cb2842f00f4e',
  'name': 'WHISKY&LIQUOR COLLECTION_CUSTOM_DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:05.000Z',
  'deleted_at': None},
 {'id': 'e9110298-cd94-483d-8dff-6240c4fcdbf1',
  'name': 'SPARKLING&SMOOTHIE',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:04.000Z',
  'deleted_at': None},
 {'id': 'e67604d4-39c4-48ba-8ae3-22571a15a746',
  'name': 'BEER&SOFT DRINK',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:03.000Z',
  'deleted_at': None},
 {'id': 'b939c9fd-bbac-4f0e-8e26-fb477adb6e9e',
  'name': 'SPARKLING WINE/BOTTLE',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:02.000Z',
  'deleted_at': None},
 {'id': '0cfced9a-f7ad-4122-a253-0eb3dfef073a',
  'name': 'MOSCATO/BOTTLE',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:01.000Z',
  'deleted_at': None},
 {'id': '691669cf-9206-407d-8e3f-73d8ba6fa32b',
  'name': 'WHITE WINE/BOTTLE',
  'color': 'GREY',
  'created_at': '2024-08-08T07:40:00.000Z',
  'deleted_at': None},
 {'id': '80c5657e-3aae-48ce-ae99-9d6f95fbf840',
  'name': 'RED WINE/BOTTLE',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:59.000Z',
  'deleted_at': None},
 {'id': 'e768b2fa-4aa3-492e-84ab-d7853431c693',
  'name': 'HOUSE WINE [ BY THE GLASS ]',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:58.000Z',
  'deleted_at': None},
 {'id': 'bb2add66-cb92-40d6-b926-e6d26ef5fa38',
  'name': 'LIQUEUR',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:57.000Z',
  'deleted_at': None},
 {'id': '55e67206-6c2c-4e06-8d40-42361e33ec59',
  'name': 'TEQUILA',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:56.000Z',
  'deleted_at': None},
 {'id': '97976a29-ec7c-4955-a38f-2a05dd909e0b',
  'name': 'VODKA',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:56.000Z',
  'deleted_at': None},
 {'id': 'd2e7a7c3-ed39-4a05-8cd0-36354017c087',
  'name': 'RUM',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:55.000Z',
  'deleted_at': None},
 {'id': 'a79fbaef-ed90-41a6-a5cf-82581de52cf6',
  'name': 'GIN',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:54.000Z',
  'deleted_at': None},
 {'id': 'fd79887b-4c6e-4af1-9ea7-e8947c318dda',
  'name': 'COGNAC & BRANDY',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:53.000Z',
  'deleted_at': None},
 {'id': '958b84c4-aa43-4198-b06c-9cdd41518503',
  'name': 'WHISKY',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:52.000Z',
  'deleted_at': None},
 {'id': '456707cb-b6b1-4845-a58c-8c92c7740118',
  'name': 'MIMOSAS',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:51.000Z',
  'deleted_at': None},
 {'id': '41b8a1c5-967f-4975-a73f-0f375baf05af',
  'name': 'FAVORITE',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:46.000Z',
  'deleted_at': None},
 {'id': '391d2cf2-3ce0-4606-8b0c-f5f28bf1ea4e',
  'name': 'CLASSIC & OTHER MARTINI',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:45.000Z',
  'deleted_at': None},
 {'id': '2dd090f9-5a23-48dc-90b4-85447097d847',
  'name': 'CLASSIC COCKTAIL',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:45.000Z',
  'deleted_at': None},
 {'id': '99246943-5a07-4498-aefb-e8d202868e12',
  'name': 'SIGNATURE MOCKTAIL',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:44.000Z',
  'deleted_at': None},
 {'id': 'ff40ed04-4d56-4d46-a9e4-eac8c24c862c',
  'name': 'SIGNATURE COCKTAIL',
  'color': 'GREY',
  'created_at': '2024-08-08T07:39:43.000Z',
  'deleted_at': None},
 {'id': '83879d35-5669-446c-86a6-434fdbd4f36a',
  'name': 'BAR_KITCHEN',
  'color': 'GREY',
  'created_at': '2024-08-04T04:59:20.000Z',
  'deleted_at': None},
 {'id': '5fcbf2fe-4d56-4e13-b4c2-520398760283',
  'name': 'BAR_DRINKS',
  'color': 'RED',
  'created_at': '2024-08-04T04:58:40.000Z',
  'deleted_at': None},
 {'id': 'e5d7514c-2fdc-4954-b195-3728eee97087',
  'name': 'Grift',
  'color': 'GREY',
  'created_at': '2024-07-08T04:41:31.000Z',
  'deleted_at': None},
 {'id': 'e3772cd2-214d-4b29-9279-71c1deb2215d',
  'name': 'SPLIT ORDERS',
  'color': 'BLUE',
  'created_at': '2024-07-07T09:06:26.000Z',
  'deleted_at': None},
 {'id': 'dc381a96-eedf-420a-84b3-c78aee13be91',
  'name': 'COLD DRINK',
  'color': 'GREEN',
  'created_at': '2024-07-05T08:55:24.000Z',
  'deleted_at': None},
 {'id': '3e55f2ba-7526-4a17-b146-04c93b88a789',
  'name': 'HOT DRINK',
  'color': 'GREEN',
  'created_at': '2024-07-05T08:55:18.000Z',
  'deleted_at': None},
 {'id': '6eda43d9-b8af-4e29-88ef-0aaee16c77c8',
  'name': 'SATURDAY SPECIAL',
  'color': 'ORANGE',
  'created_at': '2024-06-22T17:38:17.000Z',
  'deleted_at': None},
 {'id': 'd4518bfd-2379-4dbe-8ac7-dca82cd87a32',
  'name': 'BRUNCH',
  'color': 'ORANGE',
  'created_at': '2024-06-22T17:38:04.000Z',
  'deleted_at': None},
 {'id': 'ed0d6bd9-0ad7-4925-90a6-4e60e87c0b52',
  'name': 'BUILD YOUR OWN',
  'color': 'ORANGE',
  'created_at': '2024-06-22T17:37:58.000Z',
  'deleted_at': None},
 {'id': '754001f2-9338-4933-8d06-39c9718ee391',
  'name': 'BREAKFAST',
  'color': 'ORANGE',
  'created_at': '2024-06-22T17:37:46.000Z',
  'deleted_at': None},
 {'id': '4fd81f0d-d796-4538-b04d-6fdcd85758ee',
  'name': 'BAKERY',
  'color': 'ORANGE',
  'created_at': '2024-06-22T17:37:37.000Z',
  'deleted_at': None},
 {'id': 'd8a0cf2b-2bab-456c-8a14-6c9d834dbe10',
  'name': 'SOFT DRINK',
  'color': 'GREY',
  'created_at': '2024-06-22T17:37:30.000Z',
  'deleted_at': None},
 {'id': 'ed9a5724-b2fe-43cf-a7e8-2897c6badbad',
  'name': 'ITALIAN SODA',
  'color': 'GREEN',
  'created_at': '2024-06-22T17:37:23.000Z',
  'deleted_at': None},
 {'id': 'bbc87f14-65e0-4ce8-93b5-b25747fc212c',
  'name': 'COLD PRESSED JUICE',
  'color': 'GREEN',
  'created_at': '2024-06-22T17:37:14.000Z',
  'deleted_at': None},
 {'id': '49375346-92f7-402e-9c2c-fefcf298add2',
  'name': 'SMOOTHIES',
  'color': 'GREEN',
  'created_at': '2024-06-22T17:37:03.000Z',
  'deleted_at': None},
 {'id': 'af2cef22-376f-4f51-bc21-15081445cf5e',
  'name': 'TEA',
  'color': 'GREY',
  'created_at': '2024-06-22T17:36:45.000Z',
  'deleted_at': None}]

### Modifiers

In [ ]:
MODIFIER_URL = 'https://api.loyverse.com/v1.0/modifiers'

# Define a function to fetch modifier data
def fetch_modifier(modifier_id, MODIFIER_URL, headers):
    response = requests.get(f"{MODIFIER_URL}/{modifier_id}", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch modifier data. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return None

# Example usage
modifier_id = '986b5548-e1e3-4cd1-b427-354d51cc4875'  # Replace with your actual modifier ID
modifier_data = fetch_modifier(modifier_id, MODIFIER_URL, headers)

if modifier_data:
    print("Modifier Data:")
    print(modifier_data)
else:
    print("Failed to fetch modifier data.")

### Fetching Items

In [ ]:
response = fetch_items_by_category(API_URL, headers)

In [ ]:
response = fetch_items_by_category(API_URL, headers)

items_list = []

for i in response:
    items_list.append(i)

items_list

In [ ]:
items_list[4]

In [ ]:
import pandas as pd

items_data = pd.DataFrame(items_list)



In [ ]:
items_data.columns

In [ ]:
import os

for item in items_list:
    item_id = item['id']
    image_url = item['image_url']
    
    if image_url:
        # Download the image
        response = requests.get(image_url)
        
        if response.status_code == 200:
            # Save the image
            file_path = os.path.join('images', f"{item_id}.jpg")
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Image saved: {file_path}")
        else:
            print(f"Failed to download image for item {item_id}")
    else:
        print(f"No image URL for item {item_id}")

### Add Images

In [ ]:
!pip install azure-storage-blob


In [16]:
from azure.storage.blob import BlobServiceClient

# Azure connection string and container name (replace with your actual values)
connection_string = "DefaultEndpointsProtocol=https;AccountName=storagequlturelounges;AccountKey=BijYPgtrN8HTbc379lNl6VmBL2XTLLuoiS3kqnJOxLjnE68MfGrbxMubzAUTiJp9T87NkaQjs51U+AStyuZziw==;EndpointSuffix=core.windows.net"
container_name = "qulturecontainerstorage"

# Directory containing the images
directory_path = "images"  # Adjust this if your script is not in the root folder

# Initialize the BlobServiceClient and get a reference to the container
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    local_file_path = os.path.join(directory_path, filename)
    
    if os.path.isfile(local_file_path):  # Check if it is a file
        try:
            # Use the file name as the blob name
            blob_name = filename
            
            # Upload the file
            with open(local_file_path, "rb") as data:
                blob_client = container_client.get_blob_client(blob_name)
                blob_client.upload_blob(data)
            
            print(f"Uploaded {filename} to blob storage as {blob_name}")
        except Exception as e:
            print(f"Failed to upload {filename}: {e}")


Uploaded 5e3be9a6-797a-4562-b54f-d9aa2147f356.jpg to blob storage as 5e3be9a6-797a-4562-b54f-d9aa2147f356.jpg
Uploaded edfa6865-a3de-43a1-964c-ad7e1730a2e5.jpg to blob storage as edfa6865-a3de-43a1-964c-ad7e1730a2e5.jpg
Uploaded 477c2038-43a1-45df-ba7d-29cf934659ce.jpg to blob storage as 477c2038-43a1-45df-ba7d-29cf934659ce.jpg
Uploaded 54fa069e-9d17-4898-8680-531226e14204.jpg to blob storage as 54fa069e-9d17-4898-8680-531226e14204.jpg
Uploaded 98c69303-8ad0-4f7b-8470-763b524d3e3d.jpg to blob storage as 98c69303-8ad0-4f7b-8470-763b524d3e3d.jpg
Uploaded 0982f41b-f9d3-4cf5-82f6-3670804b8adc.jpg to blob storage as 0982f41b-f9d3-4cf5-82f6-3670804b8adc.jpg
Uploaded 76f9907c-3f86-497a-a18a-5671e612db00.jpg to blob storage as 76f9907c-3f86-497a-a18a-5671e612db00.jpg
Uploaded 9d5be061-ec9b-43d6-ab7d-c1a70212b2f7.jpg to blob storage as 9d5be061-ec9b-43d6-ab7d-c1a70212b2f7.jpg
Uploaded 850df696-e1db-443e-8027-00cfc0961d4d.jpg to blob storage as 850df696-e1db-443e-8027-00cfc0961d4d.jpg
Uploaded 1

### Get Stores

In [ ]:
import requests
import json

def fetch_store_ids(api_token):
    """
    Fetches a list of stores from the Loyverse API.

    :param api_token: The API token required for authentication.
    :return: A list of store IDs or a message indicating an error or no stores found.
    """
    API_URL = 'https://api.loyverse.com/v1.0/stores'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Content-Type': 'application/json'
    }

    # Making the GET request to the API endpoint
    response = requests.get(API_URL, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        stores_data = response.json()
        store_ids = [store['id'] for store in stores_data['stores']]  # Extract store IDs
        return store_ids if store_ids else "No stores found."
    else:
        # Handle possible errors
        return f"Failed to fetch stores. Status code: {response.status_code}, Message: {response.text}"

store_ids = fetch_store_ids(API_TOKEN)
print(store_ids)


### Get payment types

In [ ]:
import requests
import json
LOYVERSE_API_URL = 'https://api.loyverse.com/v1.0'
API_TOKEN = 'b43ae3d0241f44318cfa8f9b07de5e5e'  # Replace with your actual API token

headers = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}



In [ ]:
def fetch_payment_types(api_url, headers):
    response = requests.get(f"{api_url}/payment_types", headers=headers)
    if response.status_code == 200:
        return response.json().get('payment_types', [])
    else:
        print(f"Failed to fetch payment types. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return []

# Fetch payment types
payment_types = fetch_payment_types(LOYVERSE_API_URL, headers)
print(payment_types)


### Creating Orders

In [ ]:
def create_order(api_url, headers, order_data):
    response = requests.post(f"{api_url}/receipts", headers=headers, data=json.dumps(order_data))
    if response.status_code == 201:
        print("Order created successfully")
        return response.json()
    else:
        print(f"Failed to create order. Status code: {response.status_code}")
        print(f"Response: {response.text}")

# Manually input order information
order_data = {

    "store_id": "17d2d59c-227a-4929-92bb-fa67a57247c2",
    "line_items": [
        {
            "item_id": "5e3be9a6-797a-4562-b54f-d9aa2147f356",  # Replace with a valid item ID
            "variant_id": "b2d717ea-4722-4f78-b170-3303a3a22fa9",
            "quantity": 1
        }
    ],
    "payments": [
            {
                "amount": 10.0,  # Replace with the actual payment amount
                "payment_type_id": "149dfd50-a14f-4b94-a5bc-161bdf47aca1"  # Replace with the actual payment type
            }
        ]

}

# Create the order
order_response = create_order(LOYVERSE_API_URL, headers, order_data)
print(order_response)



### Open Orders

In [ ]:
import json
import requests

API_URL = 'https://api.loyverse.com/v1.0'
API_TOKEN = 'b43ae3d0241f44318cfa8f9b07de5e5e'  # Replace with your actual API token

# Define the headers with the API token
headers = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

def create_order(api_url, headers, order_data):
    response = requests.post(f"{api_url}/receipts", headers=headers, data=json.dumps(order_data))
    if response.status_code == 201:
        print("Order created successfully")
        return response.json()
    else:
        print(f"Failed to create order. Status code: {response.status_code}")
        print(f"Response: {response.text}")

# Manually input order information
order_data = {
    "store_id": "17d2d59c-227a-4929-92bb-fa67a57247c2",
    "line_items": [
        {
            "item_id": "5e3be9a6-797a-4562-b54f-d9aa2147f356",  # Replace with a valid item ID
            "variant_id": "b2d717ea-4722-4f78-b170-3303a3a22fa9",
            "quantity": 1
        }
    ],
    "payments": [
        {
            "amount": 0.0,  # Placeholder amount
            "payment_type_id": "149dfd50-a14f-4b94-a5bc-161bdf47aca1"  # Replace with the actual payment type
        }
    ]
}

# Create the order
order_response = create_order(API_URL, headers, order_data)
print(order_response)


In [ ]:
# Manually input order information
order_data = {
    "line_items": [
        {
            "item_id": "5e3be9a6-797a-4562-b54f-d9aa2147f356",  # Replace with a valid item ID
            "quantity": 1
        }
    ],
}

In [ ]:

# Create the order
order_response = create_order(LOYVERSE_API_URL, headers, order_data)
print(order_response)


### Create Modifiers

In [ ]:
import requests

API_URL = 'https://api.loyverse.com/v1.0/items'
MODIFIER_URL = 'https://api.loyverse.com/v1.0/modifiers'
API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token

# Define the headers with the API token
headers = {
    'Authorization': f'Bearer {API_TOKEN}'
}

In [ ]:
import requests
import json

# API details
API_URL = 'https://api.loyverse.com/v1.0/modifiers'
API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token

# Define the headers with the API token
headers = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

modifiers = {
    "Stuffed Salmon": [
        {"name": "Fries", "price": 80.00},
        {"name": "Sweet Fries", "price": 120.00},
        {"name": "Grits", "price": 180.00},
        {"name": "Grits with Cheese", "price": 215.00}
    ]
}





# Function to create or update modifiers
def create_or_update_modifiers():
    for drink, options in modifiers.items():
        print(f"Preparing modifiers for {drink}...")
        modifier_options = [
            {
                "id": None,  # Set to None to create new; specify ID to update existing
                "name": option['name'],
                "price": option['price'],
                "position": idx  # Using the index as the position for simplicity
            } for idx, option in enumerate(options)
        ]
        
        # Prepare the payload for the API request
        data = {
            "name": f"{drink} Options",
            "position": 0,
            "stores": ["17d2d59c-227a-4929-92bb-fa67a57247c2"],  # Specified store UUID
            "modifier_options": modifier_options
        }
        
        print(f"Sending request for {drink} with payload:")
        print(json.dumps(data, indent=4))
        
        # Send the POST request
        response = requests.post(API_URL, headers=headers, json=data)
        
        if response.status_code == 200:
            print(f"Successfully created/updated modifiers for {drink}")
        else:
            print(f"Failed to create/update modifiers for {drink}: {response.text}")

# Call the function
create_or_update_modifiers()


### Transform Menu

In [ ]:
import requests
import json

# API details
API_URL_CAT = 'https://api.loyverse.com/v1.0/categories'
API_URL_ITEMS = 'https://api.loyverse.com/v1.0/items'
API_TOKEN = 'b3bef0d711c940878753240ac13ddd19'  # Replace with your actual API token

# Define the headers with the API token
headers = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

def fetch_categories(api_url, headers):
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json().get('categories', [])
    else:
        print(f"Failed to fetch categories. Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return []

# Fetch categories
categories = fetch_categories(API_URL_CAT, headers)

# Create a mapping from category_id to category_name
category_map = {category['id']: category['name'] for category in categories}

def fetch_items_by_category(api_url, headers):
    all_items = []
    params = {
        'limit': 250
    }
    while True:
        response = requests.get(api_url, headers=headers, params=params)
        if response.status_code == 200:
            items = response.json().get('items', [])
            all_items.extend(items)
            cursor = response.json().get('cursor')
            if cursor:
                params['cursor'] = cursor  # Use the cursor to fetch the next page
            else:
                break
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            break
    return all_items

# Fetch items
items = fetch_items_by_category(API_URL_ITEMS, headers)

# Transform the menu data
def transform_menu_data(menu_data, category_map):
    transformed_data = []
    for item in menu_data:
        category_name = category_map.get(item.get("category_id"))
        transformed_item = {
            "item_name": item.get("item_name"),
            "description": item.get("description"),
            "category_name": category_name,  # Add category name
            "variants": []
        }
        for variant in item.get("variants", []):
            transformed_variant = {
                "variant_id": variant.get("variant_id"),
                "default_price": variant.get("default_price"),
                "stores": []
            }
            for store in variant.get("stores", []):
                transformed_store = {
                    "store_id": store.get("store_id"),
                    "price": store.get("price"),
                    "available_for_sale": store.get("available_for_sale")
                }
                transformed_variant["stores"].append(transformed_store)
            transformed_item["variants"].append(transformed_variant)
        transformed_data.append(transformed_item)
    return transformed_data

transformed_items = transform_menu_data(items, category_map)

# Save the transformed data to a JSON file
output_file_path = 'transformed_menu_with_categories.json'
with open(output_file_path, 'w') as output_file:
    json.dump(transformed_items, output_file, indent=4)

print(f"Transformed menu with categories saved to {output_file_path}")


In [ ]:
transformed_items

In [ ]:
def transform_menu_data(menu_data):
    transformed_data = []
    for item in menu_data:
        transformed_item = {
            "id": item.get("id"),
            "item_name": item.get("item_name"),
            "description": item.get("description"),
            "category_id": item.get("category_id"),
            "image_url": item.get("image_url"),
            "created_at": item.get("created_at"),
            "updated_at": item.get("updated_at"),
            "variants": []
        }
        for variant in item.get("variants", []):
            transformed_variant = {
                "variant_id": variant.get("variant_id"),
                "sku": variant.get("sku"),
                "default_price": variant.get("default_price"),
                "stores": []
            }
            for store in variant.get("stores", []):
                transformed_store = {
                    "store_id": store.get("store_id"),
                    "price": store.get("price"),
                    "available_for_sale": store.get("available_for_sale")
                }
                transformed_variant["stores"].append(transformed_store)
            transformed_item["variants"].append(transformed_variant)
        transformed_data.append(transformed_item)
    return transformed_data

transform_menu_data(items_list)
